## 1. 生命周期中各个方法的含义和作用
![image](page1.png)

---

### 1.1 onCreate：create 表示创建
- 这是 Activity 生命周期的第一个方法，也是我们在 android 开发中接触的最多的生命周期方法。
- 作用是进行 Activity 的一些初始化工作，比如使用 setContentView 加载布局，对一些控件和变量进行初始化等。
- 此时 Activity 还在后台，不可见。

### 1.2 onStart：start 表示启动
- 这是 Activity 生命周期的第二个方法。此时 Activity 已经可见了，但是还没出现在前台，我们还看不到，无法与 Activity 交互。
- 将 Activity 的初始化工作放在这也没有什么问题，放在 onCreate 中是由于官方推荐的以及我们开发的习惯。

### 1.3 onResume：resume 表示继续、重新开始
- Activity 在这个阶段已经出现在前台并且可见了。这个阶段可以打开独占设备。

### 1.4 onPause：pause 表示暂停
- 当 Activity 要跳到另一个 Activity 或应用正常退出时都会执行这个方法。
- 此时 Activity 在前台并可见，我们可以进行一些轻量级的存储数据和去初始化的工作，不能太耗时，因为在跳转 Activity 时只有当一个 Activity 执行完了 onPause 方法后另一个 Activity 才会启动，而且 android 中指定如果 onPause 在 500ms 即 0.5 秒内没有执行完毕的话就会强制关闭Activity。

### 1.5 onStop：stop 表示停止
- 此时 Activity 已经不可见了，但是 Activity 对象还在内存中，没有被销毁。这个阶段的主要工作也是做一些资源的回收工作。


### 1.6 onDestroy：destroy 表示毁灭
- 这个阶段 Activity 被销毁，不可见，我们可以将还没释放的资源释放，以及进行一些回收工作。

### 1.7 onRestart：restart 表示重新开始
- Activity 在这时可见，当用户按 Home 键切换到桌面后又切回来或者从后一个 Activity 切回前一个 Activity 就会触发这个方法。这里一般不做什么操作。

### 生命周期 demo
![image](page2.png)

---

## 2. onCreate 和 onStart 之间有什么区别？
- 可见与不可见的区别。前者不可见，后者可见。
- 执行次数的区别。onCreate 方法只在 Activity 创建时执行一次，而 onStart 方法在 Activity 的切换以及按 Home 键返回桌面再切回应用的过程中被多次调用。因此 Bundle 数据的恢复在 onStart 中进行比 onCreate 中执行更合适。
- onCreate 能做的事 onStart 其实都能做，但是 onStart 能做的事 onCreate 却未必适合做。如前文所说的，setContentView 和资源初始化在两者都能做，然而想动画的初始化在 onStart 中做比较好。

---

## 3. onStart 方法和 onResume 方法有什么区别？
- 是否在前台。onStart 方法中 Activity 可见但不在前台，不可交互，而在 onResume 中在前台。
- 职责不同，onStart 方法中主要还是进行初始化工作，而 onResume 方法，根据官方的建议，可以做开启动画和独占设备的操作。

---

## 4. onPause 方法和 onStop 方法有什么区别？
- 是否可见。onPause 时 Activity 可见，onStop 时 Activity 不可见，但 Activity 对象还在内存中。
- 在系统内存不足的时候可能不会执行 onStop 方法，因此程序状态的保存、独占设备和动画的关闭、以及一些数据的保存最好在 onPause 中进行，但要注意不能太耗时。

---

## 5.onStop 方法和 onDestroy 方法有什么区别？
- onStop 阶段 Activity 还没有被销毁，对象还在内存中，此时可以通过切换 Activity 再次回到该 Activity，而 onDestroy 阶段 Acivity 被销毁

---

## 6. 切换 Activity 时各方法的执行次序
- 为什么是这个：(A) onPause → (B) onCreate → (B) onStart → (B) onResume → (A) onStop
- 而不是这个：(A) onPause → (A) onStop → (B) onCreate → (B) onStart → (B) onResume
  - 一个 Activity 或多或少会占有系统资源，而在官方的建议中，onPause 方法将会释放掉很多系统资源，为切换 Activity 提供流畅性的保障，而不需要再等多两个阶段，这样做切换更快。
  - 按照生命周期图的表示，如果用户在切换 Activity 的过程中再次切回原 Activity，是在 onPause 方法后直接调用 onResume 方法的，这样比onPause → onStop → onRestart → onStart → onResume 要快得多。
  
---

## 7.与生命周期密切相关的 onSaveInstanceState 方法和 onRestoreInstanceState 方法在什么时候执行？
- 当 targetSdkVersion 小于 3 时，onSaveInstanceState 是在 onPause 方法中调用的，而大于 3 时是在 onStop 方法中调用的。
- onRestoreInstanceState 是在 onStart 之后、onResume 之前调用的。

---

## 8. Activity 异常生命周期
- 异常的生命周期是指 Activity 被系统回收或者当前设备的 Configuration 发生变化（一般指横竖屏切换）从而导致 Activity 被销毁重建。异常的生命周期主要分以下两种情况：
  - 1. 相关的系统配置发生改变导致 Activity 被杀死并重新创建（一般指横竖屏切换）
  - 2. 内存不足导致低优先级的 Activity 被杀死

---

### 情况1：配置改变
- 当 Activity 处于竖屏状态，如果突然旋转屏幕，由于系统配置发生了变化，在默认的情况下，Activity 会被销毁并重新创建

- 省略了 demo 相关代码，主要是在生命周期上加上 log，并且重写了 onSaveInstanceState 方法和 onRestoreInstanceState 方法
![image](page2.png)

#### onSaveInstanceState、onRestoreInstanceState 方法
- 当系统配置发生变化后，Activity 会被销毁，也就是 onPause，onStop，onDestroy 会被依次调用，同时因为 Activity 是在异常情况下销毁的，android 系统会自动调用 onSaveInstanceState 方法来保存当前 Activity 的状态信息，因此我们可以在 onSaveInstanceState 法中存储一些数据以便 Activity 重建之后可以恢复这些数据，调用时机必须在 onStop 方法之前，也就是 Activity 停止之前。

- 当 Activity 被重新创建之后，系统还会去调用 onRestoreInstanceState 方法，并把 Activity 销毁时通过 onSaveInstanceState 方法保存的 Bundle 对象作为参数同时传递给 onRestoreInstanceState 和 onCreate 方法，因此我们可以通过 onRestoreInstanceState 和 onCreate 方法来判断 Activity 是否被重新创建。

- *有点需要特别注意，onSaveInstanceState 和 onRestoreInstanceState 只有在 Activity 异常终止时才会被调用的，正常情况是不会调用这两个方法的。*

- onRestoreInstanceState 和 onCreate 方法都可以进行数据恢复，两者的区别在于，onRestoreInstanceState 方法一旦被系统回调，其参数 Bundle 一定不为空，无需额外的判断，而 onCreate 的 Bundle 却不一定有值，因为如果 Activity 是正常启动的话，Bundle 参数是不会有值的，因此我们需要额外的判断条件，当然虽说两者都可以数据恢复，但更倾向于 onRestoreInstanceState 方法。 
- 在 onSaveInstanceState 方法和 onRestoreInstanceState 方法中，android 系统会自动帮我们恢复一定的数据，如当前 Activity 的视图结构，文本框的数据，ListView 的滚动位置等，这些 View 相关的状态系统都会帮我们恢复，这是因为每个 View 也有 onSaveInstanceState 方法和 onRestoreInstanceState 方法。

---

### 情况2：内存不足导致低优先级的 Activity 被杀死
- 当系统内存不足的时候，系统就会按照一定的优先级去杀死目标 Acitivity 的进程来回收内存，并且此时 Activity 的 onSaveInstanceState 方法会被调用来存储数据，并在后续 Activity 恢复时调用 onRestoreInstanceState 方法来恢复数据

---

## 补充：Android 进程层次
- 共有 5 种优先级线程：Foreground Processes，Visible Processes，Service Processes， Background Processes， Empty Processes

![image](page3.png)

### 1.  Foreground Processes(前台进程)
- a. 处于前台正与用户交互的 activity 
- b. 与前台 activity 绑定的 service 
- c. 调用了 startForeground() 方法的 service 
- d. 正在执行 onCreate(), onStart(), 或 onDestroy() 方法的 service 
- e. 正在执行 onReceive() 方法的 BroadcastReceiver. 

### 2. Visible Processes(可视进程)
- 此时如果一个 Activity 可见但并非处于前台时，如在 Activity 中弹出了一个对话框，从而导致 Activity 可见但位于后台无法与用户交互，这个进程就可以被视为可见进程，同时我们也必须明白可见 activity 的 bound service 和 content provider 也处于可见进程状态。这同样是为了保证使用中的 activity 所依赖的进程不会被过早地杀掉。

### 3. Service Processes(服务进程)
- 如果我们通过 startService() 启动一个 service 服务，那么它被看作是一个服务进程。对于许多在后台做处理（如异步加载数据，获取耗时资源等）而没有立即成为前台服务的 app 都属于这种情况。

### 4. Background Processes(后台进程)
- 假如我们的 Activity 目前是前台进程，但是这时候，我们点 Home 键，将导致 onPause，onStop 方法被调用，我们的进程也就变成了后台进程，当然我们的后台进程并不会被立马杀死，所以这些进程会保留一段时间，直到更高优先级进程需要内存的时候才被回收，并且是按照最近最少使用顺序(最少使用的会被优先回收)。

### 5. Empty Processes(空进程)
- 在任何层次中，空进程都是最低优先级的。如果我们的进程不属于以上类别，那它就是空进程。
- 空进程是没有活跃的组件，只是出于缓存的目的而被保留（为了更加有效地使用内存而不是完全释放掉），只要 Android 系统内存需要可以随时杀掉它们。

---